# BLACKJACK game implementation - OOP Project

(Could also be implemented in R)
1.    Construct a deck of cards. 52 cards in total, A,2,3,…,J,Q,K for each suit (Spade/Diamond/Heart/Club)
2.    Shuffle cards randomly.
3.    If A can be 1 or 11; each J,Q,K equals to 10; and the numbered cards have their respective face values
Define a function to calculate the maximum value, but <=21, of a set of cards.
For example: the set ”AAAA” would have a value 14
4.    In a game of comparing card sets value (Players will always use maximum values of the set as calculated in Q3). From player 1’s point of view, he sees that
a.    He has Club 6 and Spade J (total is 16).
b.    The other player (player 2) has 2 Cards. One is shown to be Club K, while the other one is not shown.
If player 2 have decided to stay (meaning not getting any more cards), please use simulation method to find the probability of player 1 winning (player 1 set value > player 2 set value) if player 1 can get one and only one more card.
5.    Implement an algorithm of your choice (but not using existing sort functions) to sort cards by suit and rank (same suit stack together).

## Import Libraries

In [1]:
import random
import numpy as np

## Define game - classes and functions

In [2]:
# Prepare class for a Card :

class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

# method to show the card, in a form of a list in order to be able to use indexing (for lists)

# stripping out the value on the card:

    def show(self):
        return list([self.rank, self.suit]) 
    
    def __eq__(self, other):
        return self.rank == other.rank and self.suit == other.suit 
    

In [3]:
# Class for a Deck:
# creates a deck (returns shuffled deck - method shuffle() 
# and then method show() to show cards; method show should be run at the end of all other suitable methods)

class Deck:
    
    def __init__(self):
        self.cards = []  # initialise a list which we will later append to create a deck
        self.ready_deck = [] 
        self.suits = ['Spade', 'Diamond', 'Heart', 'Club']
        self.ranks = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
        self.construct()

        
    def construct(self):
        for r in self.ranks:
            for s in self.suits:
                self.cards.append(Card(r, s))
    
# function for removing certain cards from the deck
    def removeCards(self, CardsList, removecntr = False):# CardsList: a list of cards (hence a list of lists)
        if removecntr:
            for i in range(len(CardsList)):
                self.cards.remove(Card(CardsList[i][0], CardsList[i][1]))  
        else:
            pass

                
    # creates a list of lists, where the wrapper list is the deck and lists within correspond to cards
                
    def show(self): 
        for c in self.cards:
            self.ready_deck.append(c.show())
        return self.ready_deck
    
    def shuffle(self):
        for i in range(len(self.cards)-1, 0, -1): # looping through in reverse order
            j = random.randint(0,i)
            self.cards[i], self.cards[j] = self.cards[j], self.cards[i]
            
    def drawCard(self):
        return self.cards.pop()
    
    def drawCardSet(self, noCards):
        for i in range(1, noCards):
            return list(np.random.choice(self.cards, size = noCards, replace = False))
        

In [4]:
def setValue(card_set):
    
    '''
    The function calculates the value of cards for a given card_set based on their rank.
    Please see the dict object below for details on the respective values assigned to different card ranks.
    
    A can be either 1 or 11 (implemented in the code below) depending on the total sum score and conditional
    on the final value being at most 21.
    
    Hence e.g. "AAAA" (i.e. a draw of all four aces) would yield a value of 14.

    
    '''
    
    # define a mapping using a dict object
    d = dict(zip(['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'], 
             [11, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]))
    
    
    # calculates the (sum) value of a set of cards
    cs_show = list(map(lambda x:x.show(),card_set))
    sum_value = 0
    for i in range(0,len(cs_show)):
        sum_value = sum_value + d[list(map(lambda x:x.show(),card_set))[i][0]]
        
        
    # counts the number of occurances of aces ('A') in the card_set   
    counter = 0
    for i in range(0,len(cs_show)):
        counter = counter + cs_show[i].count('A')
        
        
    # adjusts the (sum) value
    while (sum_value>21 and counter!=0):
        sum_value = sum_value-10
        counter = counter-1
        
    
    # I assume in the code below that if the sum value obtained from the set of cards that\
    # does not contain any aces ('A') is larger than 21, then the sum value can still be at most 21
    # i.e. that the sum value can be at most 21 in any case
    
    return sum_value  
    

###### What we already know (filtration):
* the deck contains 48 cards ( as 4 had been already drawn)
* the cards drawn are: ["6", "Club"], ["J", "Spade"], ["K", "Club"] and one unknown card
* Consider the problem as a conditional probability; Prob(player1 wins | unknown card is x)
* This is what has already happened, Player 1 just does not know the hand of Player2, hence 
* my recommendation is to model the game as a conditional probability problem (i.e. events happenning one after the other)

In [5]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = []
        
    def draw(self, deck):
        self.hand.append(deck.drawCard())
        return self
    
    def show(self):
        for c in self.hand:
            return c.show()

In [6]:
def SinglePlay(P1_draw, P2_draw, removecntr = False, player1_name = 1, player2_name = 2):
    '''
    function performs a single run of the game
    
    '''
    
    CardsList = P1_draw + P2_draw
    deck1 = Deck()
    deck1.shuffle()
    deck1.removeCards(CardsList = CardsList, removecntr = removecntr)
    
    # first Player 2 draws the card
    Player2 = Player(player2_name)
    Player2.draw(deck1)
    P2_set = [] 
    for _ in range(len(P2_draw)):
        P2_set.append(Card(P2_draw[_][0], P2_draw[_][1]))
    P2_set.append(Player2.draw(deck1))
    
    # next player 1 draws the card
    Player1 = Player(player1_name)
    Player1.draw(deck1)
    P1_set = [] 
    for _ in range(len(P1_draw)):
        P1_set.append(Card(P1_draw[_][0], P1_draw[_][1]))
    P1_set.append(Player1.draw(deck1))
    
    return setValue(P1_set) > setValue(P2_set)
    

In [7]:
def SimulatorProb(n_iter):
    
    '''
    function performs simulation of the game a given number of times
    
    By the law of large numbers the probability of player 1 winning will converge in probability
    to a probability of the ratio of the number of how many times player1 won to the number
    the game has been played, given a sufficiently large number of repetitions the game has been played
    
    '''
    results = []
    
    for i in range(n_iter):
        temp = SinglePlay(P1_draw = [["J", "Spade"], ["6", "Club"]], P2_draw = [["K", "Club"]], removecntr = True)
        results.append(temp)
        
        probability = np.sum(results)/len(results)
    
    return probability

## Run Game

In [8]:
SimulatorProb(n_iter = 100)

0.87

In [9]:
SimulatorProb(n_iter = 10000)

0.8329

In [10]:
SimulatorProb(n_iter = 100000)

0.83283

## Sort cards (Q5)

#### Bubble sorting algorithm is not the most computationally efficient algorithm as it has a computational complexity of n^2, i.e. O(n^2)

In [14]:
def bubble_sorting(deck_list):
    '''
    
    The sorting algorithms used: bubble sort
    
    The below code sorts the cards in order according to both rank and suits, such that cards of the same
    suit are grouped together (suits in alphabetical order) and within each suit, the cards are ordered
    according to their ranks (assuming order as per below dictionary mapping, ie order of:
    ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'])

    deck_list: a list of shuffled cards
    '''
    
    # create a mapping which would be used for comparing:
    d = dict(zip(['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'], 
             [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]))
    
    # first sort by ranks:
    for iter_num in range(len(deck_list)-1,0,-1):
        for idx in range(iter_num):
            if d[deck_list[idx][0]] > d[deck_list[idx+1][0]]:
                temp = deck_list[idx]
                deck_list[idx] = deck_list[idx+1]
                deck_list[idx+1] = temp
                
    deck_list1 = deck_list
                
                
    # then, when sorted by ranks, sort by suits
    for iter_num in range(len(deck_list1)-1,0,-1):
        for idx in range(iter_num):
            if deck_list1[idx][1][0] > deck_list1[idx+1][1][0]:
                temp = deck_list1[idx]
                deck_list1[idx] = deck_list1[idx+1]
                deck_list1[idx+1] = temp            
                
    return deck_list1

#### Do sorting 

In [15]:
deck = Deck()

In [16]:
deck.shuffle()

In [17]:
deck_list = deck.show()

In [18]:
bubble_sorting(deck_list)

[['A', 'Club'],
 ['2', 'Club'],
 ['3', 'Club'],
 ['4', 'Club'],
 ['5', 'Club'],
 ['6', 'Club'],
 ['7', 'Club'],
 ['8', 'Club'],
 ['9', 'Club'],
 ['10', 'Club'],
 ['J', 'Club'],
 ['Q', 'Club'],
 ['K', 'Club'],
 ['A', 'Diamond'],
 ['2', 'Diamond'],
 ['3', 'Diamond'],
 ['4', 'Diamond'],
 ['5', 'Diamond'],
 ['6', 'Diamond'],
 ['7', 'Diamond'],
 ['8', 'Diamond'],
 ['9', 'Diamond'],
 ['10', 'Diamond'],
 ['J', 'Diamond'],
 ['Q', 'Diamond'],
 ['K', 'Diamond'],
 ['A', 'Heart'],
 ['2', 'Heart'],
 ['3', 'Heart'],
 ['4', 'Heart'],
 ['5', 'Heart'],
 ['6', 'Heart'],
 ['7', 'Heart'],
 ['8', 'Heart'],
 ['9', 'Heart'],
 ['10', 'Heart'],
 ['J', 'Heart'],
 ['Q', 'Heart'],
 ['K', 'Heart'],
 ['A', 'Spade'],
 ['2', 'Spade'],
 ['3', 'Spade'],
 ['4', 'Spade'],
 ['5', 'Spade'],
 ['6', 'Spade'],
 ['7', 'Spade'],
 ['8', 'Spade'],
 ['9', 'Spade'],
 ['10', 'Spade'],
 ['J', 'Spade'],
 ['Q', 'Spade'],
 ['K', 'Spade']]